<a href="https://colab.research.google.com/github/thavha/Regression-predict-group-5/blob/master/Predict_regression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importing liabraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc


In [7]:
#importing dataset
df = pd.read_csv('Train.csv')
df.drop(df.columns[[0, 1, 2, -2]], axis = 1, inplace = True) 
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
df.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival
0,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,745
1,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,1993
2,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,455
3,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,1341
4,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,1214


In [8]:

#df['Order No'] = df.replace('Order_No_,?' , '')
#df['Order No'] = df.Order No.replace({'Order_No_', '')
#df['Order No'] = df.replace(regex=[r'^Order_No_.$'], value='')
#df['Order No'] = df.replace(to_replace ="Order_No_", value ="") 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 25 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Platform Type                              21201 non-null  int64  
 1   Personal or Business                       21201 non-null  object 
 2   Placement - Day of Month                   21201 non-null  int64  
 3   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 4   Placement - Time                           21201 non-null  object 
 5   Confirmation - Day of Month                21201 non-null  int64  
 6   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 7   Confirmation - Time                        21201 non-null  object 
 8   Arrival at Pickup - Day of Month           21201 non-null  int64  
 9   Arrival at Pickup - Weekday (Mo = 1)       21201 non-null  int64  
 10  Arrival at Pickup - Ti

In [9]:
#Checking missing values
df.isnull().sum()

Platform Type                                    0
Personal or Business                             0
Placement - Day of Month                         0
Placement - Weekday (Mo = 1)                     0
Placement - Time                                 0
Confirmation - Day of Month                      0
Confirmation - Weekday (Mo = 1)                  0
Confirmation - Time                              0
Arrival at Pickup - Day of Month                 0
Arrival at Pickup - Weekday (Mo = 1)             0
Arrival at Pickup - Time                         0
Pickup - Day of Month                            0
Pickup - Weekday (Mo = 1)                        0
Pickup - Time                                    0
Arrival at Destination - Day of Month            0
Arrival at Destination - Weekday (Mo = 1)        0
Arrival at Destination - Time                    0
Distance (KM)                                    0
Temperature                                   4366
Precipitation in millimeters   

In [0]:
#Taking care of missing data
df = df.fillna(df.mean())

In [0]:
#encoding categorical data
#encoding independent categorical data
#from sklearn.compose import ColumnTransformer
#from sklearn.compose import make_column_transformer
#from sklearn.preprocessing import OneHotEncoder
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 4, 7, 10, 13])], remainder='passthrough')
#preprocessor = make_column_transformer( (OneHotEncoder(),[0,1,2,4,7,10,13,16,19,27]),remainder="passthrough")
#transformer = make_column_transformer( (OneHotEncoder(categories='auto'), [0,1,2,4,7,10,13,16,19,27]), remainder="passthrough" )
#x = np.array(ct.fit_transform(x))
#print(x)

In [19]:
#df['Placement - Time'] = (pd.to_datetime(df['Placement - Time'].str.strip(), format='%H:%M:%S %p').dt.strftime('%H:%M:%S'))
from datetime import datetime
data = str(df['Placement - Time'])
standard_time_format = datetime.strptime(data, "%I:%M %p")
only_in_24_hour_format = datetime.strftime(standard_time_format, "%H:%M")
df.head()

ValueError: ignored

In [0]:
#encoding categorical data
#get_dummies will transform all the categorical text data into numbers by adding a column for each distinct category
df_dummies = pd.get_dummies(df)

# Again we make sure that all the column names have underscores instead of whitespaces
df_dummies.columns = [col.replace(" ","_") for col in df_dummies.columns]

df_dummies.head()

,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Confirmation_-_Day_of_Month,Confirmation_-_Weekday_(Mo_=_1),Arrival_at_Pickup_-_Day_of_Month,Arrival_at_Pickup_-_Weekday_(Mo_=_1),Pickup_-_Day_of_Month,Pickup_-_Weekday_(Mo_=_1),Arrival_at_Destination_-_Day_of_Month,Arrival_at_Destination_-_Weekday_(Mo_=_1),Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Time_from_Pickup_to_Arrival,Personal_or_Business_Business,Personal_or_Business_Personal,Placement_-_Time_10:00:04_AM,Placement_-_Time_10:00:05_AM,Placement_-_Time_10:00:06_AM,Placement_-_Time_10:00:07_AM,Placement_-_Time_10:00:09_AM,Placement_-_Time_10:00:11_AM,Placement_-_Time_10:00:12_AM,Placement_-_Time_10:00:13_AM,Placement_-_Time_10:00:14_AM,Placement_-_Time_10:00:16_AM,Placement_-_Time_10:00:17_AM,Placement_-_Time_10:00:20_AM,Placement_-_Time_10:00:23_AM,Placement_-_Time_10:00:25_AM,Placement_-_Time_10:00:26_AM,Placement_-_Time_10:00:28_AM,Placement_-_Time_10:00:30_AM,Placement_-_Time_10:00:31_AM,Placement_-_Time_10:00:32_AM,...,Arrival_at_Destination_-_Time_9:57:57_AM,Arrival_at_Destination_-_Time_9:57:58_AM,Arrival_at_Destination_-_Time_9:58:00_AM,Arrival_at_Destination_-_Time_9:58:04_AM,Arrival_at_Destination_-_Time_9:58:07_AM,Arrival_at_Destination_-_Time_9:58:08_AM,Arrival_at_Destination_-_Time_9:58:18_AM,Arrival_at_Destination_-_Time_9:58:20_AM,Arrival_at_Destination_-_Time_9:58:23_AM,Arrival_at_Destination_-_Time_9:58:27_AM,Arrival_at_Destination_-_Time_9:58:34_AM,Arrival_at_Destination_-_Time_9:58:36_AM,Arrival_at_Destination_-_Time_9:58:37_AM,Arrival_at_Destination_-_Time_9:58:38_AM,Arrival_at_Destination_-_Time_9:58:43_AM,Arrival_at_Destination_-_Time_9:58:44_AM,Arrival_at_Destination_-_Time_9:58:47_AM,Arrival_at_Destination_-_Time_9:58:49_AM,Arrival_at_Destination_-_Time_9:58:50_AM,Arrival_at_Destination_-_Time_9:58:52_AM,Arrival_at_Destination_-_Time_9:58:56_AM,Arrival_at_Destination_-_Time_9:59:02_AM,Arrival_at_Destination_-_Time_9:59:04_AM,Arrival_at_Destination_-_Time_9:59:11_AM,Arrival_at_Destination_-_Time_9:59:21_AM,Arrival_at_Destination_-_Time_9:59:23_AM,Arrival_at_Destination_-_Time_9:59:26_AM,Arrival_at_Destination_-_Time_9:59:30_AM,Arrival_at_Destination_-_Time_9:59:33_AM,Arrival_at_Destination_-_Time_9:59:36_AM,Arrival_at_Destination_-_Time_9:59:37_AM,Arrival_at_Destination_-_Time_9:59:38_AM,Arrival_at_Destination_-_Time_9:59:42_AM,Arrival_at_Destination_-_Time_9:59:43_AM,Arrival_at_Destination_-_Time_9:59:45_AM,Arrival_at_Destination_-_Time_9:59:46_AM,Arrival_at_Destination_-_Time_9:59:49_AM,Arrival_at_Destination_-_Time_9:59:55_AM,Arrival_at_Destination_-_Time_9:59:56_AM,Arrival_at_Destination_-_Time_9:59:57_AM
0,3,9,5,9,5,9,5,9,5,9,5,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,12,5,12,5,12,5,12,5,12,5,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,1993,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,30,2,30,2,30,2,30,2,30,2,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,455,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,15,5,15,5,15,5,15,5,15,5,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,1341,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,13,1,13,1,13,1,13,1,13,1,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,1214,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_dummies.shape

(21201, 78631)

In [0]:
#reorder columns so that our dependent variable is the last column of the dataframe
column_titles = [col for col in df_dummies.columns if col!= 'Time_from_Pickup_to_Arrival'] + ['Time_from_Pickup_to_Arrival']
df_dummies=df_dummies.reindex(columns=column_titles)

In [0]:
#df_dummies.corr()

In [0]:
from statsmodels.formula.api import ols

# Model DataFrame with all of the columns:
dfm = df_dummies.copy()

# The dependent variable:
y_name = 'Time_from_Pickup_to_Arrival'
# The independent variable
# (let's first try all of the columns in the model DataFrame)
X_names = [col for col in dfm.columns if col != y_name]

# Build the OLS formula string " y ~ X "
formula_str = y_name+" ~ "+" + ".join(X_names);
print('Formula:\n\t {}'.format(formula_str))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Formula:
	 Time_from_Pickup_to_Arrival ~ Platform_Type + Placement_-_Day_of_Month + Placement_-_Weekday_(Mo_=_1) + Confirmation_-_Day_of_Month + Confirmation_-_Weekday_(Mo_=_1) + Arrival_at_Pickup_-_Day_of_Month + Arrival_at_Pickup_-_Weekday_(Mo_=_1) + Pickup_-_Day_of_Month + Pickup_-_Weekday_(Mo_=_1) + Arrival_at_Destination_-_Day_of_Month + Arrival_at_Destination_-_Weekday_(Mo_=_1) + Distance_(KM) + Temperature + Precipitation_in_millimeters + Pickup_Lat + Pickup_Long + Destination_Lat + Destination_Long + Personal_or_Business_Business + Personal_or_Business_Personal + Placement_-_Time_10:00:04_AM + Placement_-_Time_10:00:05_AM + Placement_-_Time_10:00:06_AM + Placement_-_Time_10:00:07_AM + Placement_-_Time_10:00:09_AM + Placement_-_Time_10:00:11_AM + Placement_-_Time_10:00:12_AM + Placement_-_Time_10:00:13_AM + Placement_-_Time_10:00:14_AM + Placement_-_Time_10:00:16_AM + Placement_-_Time_10:00:17_AM + Placement_-_Time_10:00:20_AM + Placement_-_Time_10:00:23_AM + Placement_-_Time_10

In [0]:
# Fit the model using the model dataframe
model=ols(formula=formula_str, data=dfm)
fitted = model.fit()

# Output the fitted summary
print(fitted.summary())

In [0]:
df_dummies = pd.get_dummies(df, drop_first=True)

# Again make sure that all the column names have underscores instead of whitespaces
df_dummies.columns = [col.replace(" ", "_") for col in df_dummies.columns]

# Reorder columns with the dependent variable (claim_amount) the last column
column_titles = [col for col in df_dummies.columns if col !=
                 'Time_from_Pickup_to_Arrival'] + ['Time_from_Pickup_to_Arrival']
df_dummies = df_dummies.reindex(columns=column_titles)

df_dummies.head()

,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Confirmation_-_Day_of_Month,Confirmation_-_Weekday_(Mo_=_1),Arrival_at_Pickup_-_Day_of_Month,Arrival_at_Pickup_-_Weekday_(Mo_=_1),Pickup_-_Day_of_Month,Pickup_-_Weekday_(Mo_=_1),Arrival_at_Destination_-_Day_of_Month,Arrival_at_Destination_-_Weekday_(Mo_=_1),Distance_(KM),Temperature,Precipitation_in_millimeters,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Personal_or_Business_Personal,Placement_-_Time_10:00:05_AM,Placement_-_Time_10:00:06_AM,Placement_-_Time_10:00:07_AM,Placement_-_Time_10:00:09_AM,Placement_-_Time_10:00:11_AM,Placement_-_Time_10:00:12_AM,Placement_-_Time_10:00:13_AM,Placement_-_Time_10:00:14_AM,Placement_-_Time_10:00:16_AM,Placement_-_Time_10:00:17_AM,Placement_-_Time_10:00:20_AM,Placement_-_Time_10:00:23_AM,Placement_-_Time_10:00:25_AM,Placement_-_Time_10:00:26_AM,Placement_-_Time_10:00:28_AM,Placement_-_Time_10:00:30_AM,Placement_-_Time_10:00:31_AM,Placement_-_Time_10:00:32_AM,Placement_-_Time_10:00:33_AM,Placement_-_Time_10:00:35_AM,Placement_-_Time_10:00:38_AM,...,Arrival_at_Destination_-_Time_9:57:58_AM,Arrival_at_Destination_-_Time_9:58:00_AM,Arrival_at_Destination_-_Time_9:58:04_AM,Arrival_at_Destination_-_Time_9:58:07_AM,Arrival_at_Destination_-_Time_9:58:08_AM,Arrival_at_Destination_-_Time_9:58:18_AM,Arrival_at_Destination_-_Time_9:58:20_AM,Arrival_at_Destination_-_Time_9:58:23_AM,Arrival_at_Destination_-_Time_9:58:27_AM,Arrival_at_Destination_-_Time_9:58:34_AM,Arrival_at_Destination_-_Time_9:58:36_AM,Arrival_at_Destination_-_Time_9:58:37_AM,Arrival_at_Destination_-_Time_9:58:38_AM,Arrival_at_Destination_-_Time_9:58:43_AM,Arrival_at_Destination_-_Time_9:58:44_AM,Arrival_at_Destination_-_Time_9:58:47_AM,Arrival_at_Destination_-_Time_9:58:49_AM,Arrival_at_Destination_-_Time_9:58:50_AM,Arrival_at_Destination_-_Time_9:58:52_AM,Arrival_at_Destination_-_Time_9:58:56_AM,Arrival_at_Destination_-_Time_9:59:02_AM,Arrival_at_Destination_-_Time_9:59:04_AM,Arrival_at_Destination_-_Time_9:59:11_AM,Arrival_at_Destination_-_Time_9:59:21_AM,Arrival_at_Destination_-_Time_9:59:23_AM,Arrival_at_Destination_-_Time_9:59:26_AM,Arrival_at_Destination_-_Time_9:59:30_AM,Arrival_at_Destination_-_Time_9:59:33_AM,Arrival_at_Destination_-_Time_9:59:36_AM,Arrival_at_Destination_-_Time_9:59:37_AM,Arrival_at_Destination_-_Time_9:59:38_AM,Arrival_at_Destination_-_Time_9:59:42_AM,Arrival_at_Destination_-_Time_9:59:43_AM,Arrival_at_Destination_-_Time_9:59:45_AM,Arrival_at_Destination_-_Time_9:59:46_AM,Arrival_at_Destination_-_Time_9:59:49_AM,Arrival_at_Destination_-_Time_9:59:55_AM,Arrival_at_Destination_-_Time_9:59:56_AM,Arrival_at_Destination_-_Time_9:59:57_AM,Time_from_Pickup_to_Arrival
0,3,9,5,9,5,9,5,9,5,9,5,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,745
1,3,12,5,12,5,12,5,12,5,12,5,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1993
2,3,30,2,30,2,30,2,30,2,30,2,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,455
3,3,15,5,15,5,15,5,15,5,15,5,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1341
4,1,13,1,13,1,13,1,13,1,13,1,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1214


In [0]:
# We'll keep the model DataFrame, but only specify the columns we want to fit this time
X_names = [col for col in df_dummies.columns if col != y_name]

# Build the OLS formula string " y ~ X "
formula_str = y_name+' ~ '+'+'.join(X_names)

# Fit the model using the model dataframe
model = ols(formula=formula_str, data=dfm)
fitted = model.fit()

# Output the fitted summary
print(fitted.summary())

In [0]:
import sys
sys.setrecursionlimit(10**6) 

In [0]:
from scipy.stats import pearsonr

# Build a dictionary of correlation coefficients and p-values
dict_cp = {}

column_titles = [col for col in corrs.index if col!= 'Time_from_Pickup_to_Arrival']
for col in column_titles:
    p_val = round(pearsonr(df_dummies[col], df_dummies['Time_from_Pickup_to_Arrival'])[1],6)
    dict_cp[col] = {'Correlation_Coefficient':corrs[col],
                    'P_Value':p_val}

df_cp = pd.DataFrame(dict_cp).T
df_cp_sorted = df_cp.sort_values('P_Value')
df_cp_sorted[df_cp_sorted['P_Value']<0.1]

In [0]:
# Keep only the variables that have a significant correlation with the dependent variable
# The dependent variable remains the same:
y_data = df_dummies[y_name]  # y_name = 'Time_from_Pickup_to_Arrival'

# Model building - Independent Variable (IV) DataFrame
X_names = list(df_cp[df_cp['P_Value'] < 0.05].index)
X_data = df_dummies[X_names]

In [0]:
# Create the correlation matrix
corr = X_data.corr()

# Find rows and columnd where correlation coefficients > 0.9 or <-0.9
corr[np.abs(corr) > 0.9]

In [0]:
# As before, we create the correlation matrix
# and find rows and columnd where correlation coefficients > 0.9 or <-0.9
corr = X_data.corr()
r, c = np.where(np.abs(corr) > 0.9)

# We are only interested in the off diagonal entries:
off_diagonal = np.where(r != c)

# Show the correlation matrix rows and columns where we have highly correlated off diagonal entries:
corr.iloc[r[off_diagonal], c[off_diagonal]]

In [0]:
#splitting dataset
# import train/test split module
from sklearn.model_selection import train_test_split
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.20,
                                                    random_state=1)

In [0]:
#training a model
# Fitting Regression to the dataset
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [0]:
# Get training and testing data for variance threshold model
X_var_train = X_train[X_var_names]
X_var_test = X_test[X_var_names]

In [0]:
# Get training and testing data for correlation threshold model
X_corr_train = X_train[X_corr_names]
X_corr_test = X_test[X_corr_names]

In [0]:
#Fit models
lm = LinearRegression()
lm_corr = LinearRegression()
lm_var = LinearRegression()


In [0]:
lm.fit(X_train, y_train);
lm_corr.fit(X_corr_train,y_train);
lm_var.fit(X_var_train,y_train);

In [0]:
# Assess model accuracy
from sklearn.metrics import mean_squared_error, r2_score
# Create figure and axes
f, ax = plt.subplots(figsize=(15, 5), nrows=1, ncols=3, sharey=True)

# Create list of titles and predictions to use in for loop
train_pred = [lm.predict(X_train),
              lm_corr.predict(X_corr_train),
              lm_var.predict(X_var_train)]
test_pred = [lm.predict(X_test),
             lm_corr.predict(X_corr_test),
             lm_var.predict(X_var_test)]
title = ['No threshold', 'Corr threshold', 'Var threshold']

# Key:
# No threshold - linear regression with all predictive variables
# Corr threshold - linear regression with correlation thresholded predictive variables
# Var threshold - linear regression with variance thresholded predictive variables


# Loop through all axes to plot each model's results
for i in range(3):
    test_mse = round(mean_squared_error(test_pred[i], y_test), 4)
    test_r2 = round(r2_score(test_pred[i], y_test), 4)
    train_mse = round(mean_squared_error(train_pred[i], y_train), 4)
    train_r2 = round(r2_score(train_pred[i], y_train), 4)
    title_str = f"Linear Regression({title[i]}) \n train MSE = {train_mse} \n " + \
                f"test MSE = {test_mse} \n training $R^{2}$ = {train_r2} \n " + \
                f"test $R^{2}$ = {test_r2}"
    ax[i].set_title(title_str)
    ax[i].set_xlabel('Actual')
    ax[i].set_ylabel('Predicted')
    ax[i].plot(y_test, y_test, 'r')
    ax[i].scatter(y_test, test_pred[i])


In [0]:
new_df = df[['User Id','Platform Type','Personal or Business','Placement - Day of Month','Placement - Weekday (Mo = 1)','Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Arrival at Destination - Time','Distance (KM)','Temperature','Precipitation in millimeters','Rider Id','Time from Pickup to Arrival']].copy()